In [1]:
import pandas as pd

In [2]:
df=pd.read_csv('../230817_intent_labeled_by_6.csv')

In [3]:
# df.drop(columns={'label','intent'})
# self.labels = {
#                 0:"주문", 1:"배송", 2:"매장", 3:"AS", 
# 4:"제품_정보", 5:"제품_재고"
#             }
df

,query,intent,label
0,아침도 아니고 밤 12시 30분에 결제한 건데도 그런가요?,주문,0.0
1,실수로 취소하면 재주문해야 하는 거죠?,주문,0.0
2,택배비 따로 추가되나요?,배송,1.0
3,택배비 있나요?,배송,1.0
4,택배비 따로 들어요,배송,1.0
...,...,...,...
79612,맨투맨 3가지 사이즈로만 나왔어요?,제품,4.0
79613,사이즈가 80A 인데 윗가슴 부분이 들뜨지 않나요?,제품,4.0
79614,사이즈가 29 - 30 정도면 라지인가요?,제품,4.0
79615,혹시나 M사이즈는 없죠?,제품,4.0


In [4]:
df['id'] = range(1, len(df) + 1)

In [31]:
import re

# 주어진 규칙에 따라 태그를 결정하는 함수를 정의한다
def tag_word(id, word, label):
    """
    문장 ID, 단어, 라벨을 입력으로 받아서 해당 단어에 부여할 태그를 결정한다.
    """
    # 1-1. XX시, YY분 형식일 경우 B-TI를 부여한다
    if re.search(r'\d+시|\d+분', word):
        return 'B-TI'
    
    time = ['아침', '점심', '저녁', '밤', '새벽', '자정']
    if word in time:
        return 'B-TI'
    
    # 1-2. 레오롱스커트, 맨투맨, 청바지는 B-PRODUCT를 부여한다
    if word in ['레오롱스커트', '맨투맨', '청바지']:
        return 'B-PRODUCT'
    
    # 1-3. 지역명에 대해 B-LOCATION을 부여한다
    locations = ['서울', '인천', '경기', '강원', '충북', '충남', '충청', '세종', 
                 '대전', '전남', '전북', '광주', '전라', '경북', '경남', '경상', 
                 '부산', '울산', '대구']
    if any(loc in word for loc in locations):
        return 'B-LOCATION'
    
    # 2, 3. 4번 라벨과 5번 라벨에 대한 규칙
    if label == 4.0 or label == 5.0:
        if re.search(r'\d+[A-Za-z]+|\d+사이즈|\d+', word):
            return 'B-SIZE'
    
    # 1-4. 위의 규칙에 해당하지 않는 단어는 'O'를 부여한다
    return 'O'


def apply_tag(row):
    return tag_word(row['id'], row['word'], row['label'])


In [6]:
df


,query,intent,label,id
0,아침도 아니고 밤 12시 30분에 결제한 건데도 그런가요?,주문,0.0,1
1,실수로 취소하면 재주문해야 하는 거죠?,주문,0.0,2
2,택배비 따로 추가되나요?,배송,1.0,3
3,택배비 있나요?,배송,1.0,4
4,택배비 따로 들어요,배송,1.0,5
...,...,...,...,...
79612,맨투맨 3가지 사이즈로만 나왔어요?,제품,4.0,79613
79613,사이즈가 80A 인데 윗가슴 부분이 들뜨지 않나요?,제품,4.0,79614
79614,사이즈가 29 - 30 정도면 라지인가요?,제품,4.0,79615
79615,혹시나 M사이즈는 없죠?,제품,4.0,79616


In [20]:
from ai.myapp.chatbot.utils.Preprocess import *

p = Preprocess(userdic='/home/azureuser/projects/aerius/ai/myapp/chatbot/dict/userdic.txt')

id_list=df['id'].tolist()
label_list=df['label'].tolist()
query_list=df['query'].tolist()

def create_word_id_list(id_list, query_list, label_list):
    word_id_list = []
    
    for i, sentence in enumerate(query_list):
        # 문장을 단어 단위로 분리한다
        pp=p.delete_intent_trash_tags(sentence)
        word_list, _  = p.divide_words_tags(pp)
        
        # 각 단어와 그 단어가 속한 문장의 ID를 튜플 형태로 저장한다
        for word in word_list:
            word_id_list.append((word, id_list[i], label_list[i]))
    
    return word_id_list

# 문장 ID와 문장 리스트를 기반으로 단어와 문장 ID를 리스트에 저장한다
word_id_list = create_word_id_list(id_list, query_list, label_list)

# 결과 중 앞부분만 확인
sample_list_result = word_id_list[:10]
sample_list_result


# new_df = []
# for sentence in query_list:
#     pp=p.delete_intent_trash_tags(sentence)
#     new_df.append(pp)

# new_df[:5]

# words = []
# for preprocessed in new_df:
#     word_list, _ = p.divide_words_tags(preprocessed)
#     words.extend(word_list)

# words[:5]


/home/azureuser/projects/aerius/ai/myapp/chatbot/dict/userdic.txt


[('아침', 1, 0.0),
 ('아니', 1, 0.0),
 ('고', 1, 0.0),
 ('밤', 1, 0.0),
 ('12시', 1, 0.0),
 ('30분', 1, 0.0),
 ('결제', 1, 0.0),
 ('걸', 1, 0.0),
 ('ㄴ데', 1, 0.0),
 ('그렇', 1, 0.0)]

In [23]:
sample_list_result = word_id_list[-100:]
sample_list_result

[('시키', 79609, 4.0),
 ('걸', 79609, 4.0),
 ('ㄴ가요', 79609, 4.0),
 ('?', 79609, 4.0),
 ('사이즈', 79610, 4.0),
 ('빅사', 79610, 4.0),
 ('이즈', 79610, 4.0),
 ('이', 79610, 4.0),
 ('ㄴ가요', 79610, 4.0),
 ('?', 79610, 4.0),
 ('09', 79611, 4.0),
 ('번', 79611, 4.0),
 ('의', 79611, 4.0),
 ('밴', 79611, 4.0),
 ('딩', 79611, 4.0),
 ('일자', 79611, 4.0),
 ('팬츠', 79611, 4.0),
 ('XL', 79611, 4.0),
 ('허리', 79611, 4.0),
 ('사이즈', 79611, 4.0),
 ('몇', 79611, 4.0),
 ('인치', 79611, 4.0),
 ('이', 79611, 4.0),
 ('ㄴ지요', 79611, 4.0),
 ('?', 79611, 4.0),
 ('4', 79612, 4.0),
 ('베이직', 79612, 4.0),
 ('9', 79612, 4.0),
 ('부', 79612, 4.0),
 ('스판', 79612, 4.0),
 ('슬랙스', 79612, 4.0),
 ('사이즈', 79612, 4.0),
 ('L', 79612, 4.0),
 ('허리', 79612, 4.0),
 ('사이즈', 79612, 4.0),
 ('33', 79612, 4.0),
 ('되', 79612, 4.0),
 ('어', 79612, 4.0),
 ('있', 79612, 4.0),
 ('는데', 79612, 4.0),
 ('맞', 79612, 4.0),
 ('?', 79612, 4.0),
 ('맨투맨', 79613, 4.0),
 ('3', 79613, 4.0),
 ('가지', 79613, 4.0),
 ('사이즈', 79613, 4.0),
 ('나오', 79613, 4.0),
 ('았', 79613, 4.0),
 ('

In [28]:
# DataFrame 생성
new_df = pd.DataFrame(word_id_list, columns=['word', 'id', 'label'])

In [29]:
new_df

,word,id,label
0,아침,1,0.0
1,아니,1,0.0
2,고,1,0.0
3,밤,1,0.0
4,12시,1,0.0
...,...,...,...
1000916,알,79617,4.0
1000917,수,79617,4.0
1000918,있,79617,4.0
1000919,을까요,79617,4.0


In [1]:
new_df['tag'] = new_df.apply(apply_tag, axis=1)

# 결과 중 앞부분만 확인
new_df.head(6)

NameError: name 'new_df' is not defined

In [34]:
new_df.to_csv('./230822_tagging.csv', index=False)